In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

In [ ]:
# make page
paper_size = '5x7 inches'
border:float=20
paper = Paper('7x5 inches')

drawbox = paper.get_drawbox(border)

In [ ]:
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=35, ystep=30, flatmesh=True)

In [ ]:
points = [Point(x,y) for x,y in itertools.product(xs,ys)]

In [ ]:
point = Point((0,0))
rp = 

In [ ]:
gp.RegPolygon(point=point, radius=1, n_corners=8).corners

In [ ]:
p = point.buffer(np.random.uniform(3, 5))
stp = gp.ScaleTransPrms(d_buffer=-0.5,angles=-45,d_translate_factor=0.)
P = gp.Poly(p)
P.fill_scale_trans(**stp.prms)

buffer_factor = 1e-4
cs=2
js=1
r = 4
mfills = []
for f in P.fill:
    dilated = f.buffer(buffer_factor, cap_style=cs, join_style=js, resolution=r)
    eroded = dilated.buffer(-buffer_factor, cap_style=cs, join_style=js, resolution=r)
    mfills.append(eroded)

mfills = gp.merge_Polygons(mfills)
mfills.buffer(0.2, cap_style=2, join_style=1, resolution=4).boundary

In [ ]:
mazes = []
for point in points:
    p = point.buffer(np.random.uniform(8, 16))
    stp = gp.ScaleTransPrms(d_buffer=-3,angles=-45,d_translate_factor=0.)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)

    buffer_factor = 1e-4
    cs=2
    js=1
    r = 4
    mfills = []
    for f in P.fill:
        dilated = f.buffer(buffer_factor, cap_style=cs, join_style=js, resolution=r)
        eroded = dilated.buffer(-buffer_factor, cap_style=cs, join_style=js, resolution=r)
        mfills.append(eroded)

    mfills = gp.merge_Polygons(mfills)
    buffs = mfills.buffer(0.2, cap_style=2, join_style=1, resolution=4).boundary
    filtered_buffs = gp.merge_LineStrings([f for f in buffs if f.length>0.1])
    mazes.append(filtered_buffs)
mazes = gp.merge_LineStrings(mazes)

In [ ]:
filtered_buffs

In [ ]:
layers = []
layers.append(mazes)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# tolerance=0.5
# sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()